## extract selected points and combine scans 

In [ ]:
import os
import sys
import glob
import pandas as pd
import ply_io

In [5]:
project = '2021-06-28.Ove_W2-Fc(new).riproject/'
name = os.path.split(project)[1]
print ('processing project:', name)

df = pd.DataFrame()

for asc in glob.glob(os.path.join(project, 'ascii', '*.ascii')):

    scan_position = int(os.path.split(asc)[-1][7:10])
    print ('\tprocessing scan position:', scan_position)

    # read in scan
    tmp = pd.read_csv(asc, engine='c', dtype=float, error_bad_lines=False)
    tmp.columns = ['PID[]','XYZ[0][m]','XYZ[1][m]','XYZ[2][m]','Deviation[]','Reflectance[dB]','Selected[]']
    column2column = {u'PID[]':'pid', u'Target Count[]':'tot_rtn', u'XYZ[0][m]':'x', u'XYZ[1][m]':'y', u'XYZ[2][m]':'z',
                     u'Deviation[]':'dev', u'Reflectance[dB]':'refl', u'Target Index[]':'rtn_N', u'Selected[]':'sel'}
    tmp.columns = tmp.columns.map(column2column)
    tmp = tmp[tmp.sel == 1]
    if len(tmp) == 0:
        raise Exception('len df == 0, were required points selected in RiSCAN')
    tmp.loc[:, 'sp'] = scan_position

    # calc distance
    #         mat = np.loadtxt('../matrix/ScanPos00{}.DAT'.format(scan_position))
    #         tmp.loc[:, 'rng'] = np.linalg.norm(tmp[['x', 'y', 'z']] - mat[:3, 3], axis=1)

    df = df.append(tmp)#[tmp.refl.between(-20, 10)])

processing project: 
	processing scan position: 1
	processing scan position: 2
	processing scan position: 3
	processing scan position: 4


In [6]:
ply_io.write_ply(os.path.join(project, 'ascii',  name + '.ply'),  
                 df[[c for c in df.columns if isinstance(c, str)]])

In [8]:
df

,pid,x,y,z,dev,refl,sel,sp
4532169,9880406.0,-0.26751,-1.06231,-0.99242,9.0,-4.64,1.0,1
4532171,9889137.0,-0.26801,-1.06406,-0.99267,10.0,-4.38,1.0,1
4533244,10050538.0,-0.29222,-1.08609,-0.98692,7.0,-5.02,1.0,1
4533251,10059249.0,-0.29147,-1.08484,-0.98467,10.0,-5.05,1.0,1
4533252,10050536.0,-0.29072,-1.08484,-0.98592,10.0,-4.70,1.0,1
...,...,...,...,...,...,...,...,...
16911298,29005409.0,0.88867,-1.36887,0.87515,4.0,-10.02,1.0,4
16911299,28995730.0,0.88837,-1.36982,0.87436,3.0,-9.86,1.0,4
16911300,29000568.0,0.88841,-1.36927,0.87487,5.0,-9.36,1.0,4
16911301,28990874.0,0.88763,-1.37050,0.87403,3.0,-9.51,1.0,4


## extract branch 

## filter 

In [1]:
import qrdar

In [7]:
import pandas as pd
path = '2021-06-28.Ove_W2-Fc(new).riproject/qrdar.csv'
translation = pd.read_csv(path)

In [8]:
translation

,project name code
0,2021-06-28.Ove_W2-Fc(new) Ove_W2_Fc 28


In [5]:
ctag = lambda row: '{}-{}-{}'.format(*row[['plot', 'treetag', 'light']])
ctag

<function __main__.<lambda>(row)>

In [9]:
translation.rename(columns={c:c.lower() for c in translation.columns}, inplace=True)
translation

,project name code
0,2021-06-28.Ove_W2-Fc(new) Ove_W2_Fc 28


In [10]:
translation.loc[:, 'tag'] = translation.apply(ctag, axis=1)

KeyError: "None of [Index(['plot', 'treetag', 'light'], dtype='object')] are in the [index]"

In [ ]:
translation.tag = [t.replace('-nan', '') for t in translation.tag]
translation = translation[translation.project == project]